In [ ]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics

import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Hyperplane,get_Electricity,get_AGRAWL,get_Airlines,get_CoverType,get_RandomRBF
from train_and_eval import train_and_evaluate
from functools import partial
from train_and_eval import train_and_evaluate
from river.metrics.accuracy import Accuracy
from river.metrics import CohenKappa
from multiprocessing.pool import ThreadPool
from river.datasets.synth import ConceptDriftStream
from tqdm import tqdm

# methods
from river.dummy import NoChangeClassifier,PriorClassifier
from river.cluster import CluStream
from river.forest.adaptive_random_forest import ARFClassifier
from river.tree.hoeffding_adaptive_tree_classifier import  HoeffdingAdaptiveTreeClassifier
from river.neighbors import KNNClassifier as riverKNN
from river.naive_bayes import GaussianNB
from semisupervised_methods.clustream_and_label import CluserAndLabel
from semisupervised_methods.incremental_classifier import IncrementalClassifer
from semisupervised_methods.one_nearest_neighbour import oneNNClassifer
from river.drift import ADWIN, PageHinkley


# Classifers


## Dummies

In [2]:
NoChange = NoChangeClassifier
Majority = PriorClassifier

## Fully supervised

In [3]:
HTAdaptive = HoeffdingAdaptiveTreeClassifier #TODO: if no detector given does it work like the one without the Adaptive
AdaptiveRF = ARFClassifier
KNN= riverKNN
NaiveBayes = GaussianNB

## Semi supervised

In [4]:
#TODO: OneNN -> just for binary classification _> also can check for flipped classes
Incremental = IncrementalClassifer # more like a wrapper
CluserLabel = CluserAndLabel

## Drift detection

In [5]:
Adwin = ADWIN
PH = PageHinkley
# Image base - ask if I should implement

# Notebook

In [6]:
from river.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
from river.cluster.clustream import CluStream
clst = CluStream(n_macro_clusters=5)

In [18]:
clst = CluserAndLabel(max_micro_clusters=10)
led_stream = get_Electricity()
scaler = StandardScaler()
for i,(x,y) in enumerate(led_stream):
    scaler = scaler.learn_one(x)
    x = scaler.transform_one(x)
    if i <9990:
        # if y ==7:
        #     print(clustream.return_microclusters() )
        if i %10 == 9:
            clst = clst.learn_one(x,y)
        else:
            clst = clst.learn_one(x = x,y = y)
        #print(clustream.sum_labels())
        print([a.labels for a in clst.micro_clusters.values()])
    else:
        print(1)
        break

[{1: 1}]
[{1: 1}, {1: 1}]
[{1: 1}, {1: 1}, {1: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1, 1: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1, 1: 1}, {0: 1}, {0: 1}, {1: 1}, {0: 2}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1, 1: 1}, {0: 1}, {0: 1}, {1: 2}, {0: 2}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1}, {0: 2, 1: 1}, {0: 1}, {1: 2}, {0: 2}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1, 1: 1}, {0: 2, 1: 1}, {0: 1}, {1: 2}, {0: 2}, {0: 1}]
[{1: 1}, {1: 1}, {1: 1}, {1: 1}, {0: 1, 1: 2}, {0: 2, 1: 1}, {0: 1}, {1: 2}, {0: 2}, {0: 1}]
[{1: 1}, {1: 1},

In [59]:
clst.micro_clusters[cluster_num].labels

{1: 2}

In [6]:
stream = generate_stream_section(
        get_Hyperplane(), f'L','HyperPlane',0,10000)

In [9]:
for i,(x,y) in enumerate(led_stream):  
    if i <99:
        print(y, clst.predict_one(x))
    else:
        break

0 1
1 0
1 0
0 1
1 0
0 1
0 1
1 1
1 1
0 0
0 1
0 1
1 0
1 1
0 0
0 1
0 0
1 0
0 1
1 1
0 1
0 1
0 1
1 1
1 0
0 1
0 0
0 1
1 0
1 0
1 0
0 1
1 0
1 1
0 1
0 1
0 1
0 1
1 4
0 1
1 0
0 1
0 0
1 0
1 0
0 4
1 1
0 1
0 1
0 1
1 0
1 1
0 1
1 1
1 1
1 1
0 1
0 1
0 1
1 1
0 1
1 0
1 0
0 1
0 1
0 0
0 1
0 1
0 0
0 1
0 1
1 0
0 0
0 1
1 1
0 1
1 0
1 0
0 1
0 1
1 0
0 0
1 0
0 1
1 0
1 4
1 0
0 0
1 0
1 1
0 1
1 4
1 4
1 1
0 0
1 0
0 0
1 0
0 1


In [22]:
last = -1
s = 0
for i,y in pd.DataFrame.from_records([y for y in get_Electricity()])[1].items():
    if last==y:
        s+=1
    last=y

In [25]:
np.mean([y for x,y in get_Electricity()])

0.4245453742937853

In [24]:
s/len(get_Electricity())

0.8532838983050848

In [2]:
hp =  get_Electricity()
dataset_name = 'Electricity'
Q=[(0,1000)]
probas = [0.1,0.2,.5]
clustream = CluserAndLabel
no_change = NoChangeClassifier
HT = HoeffdingAdaptiveTreeClassifier
incClasif2= IncrementalClassifer
incClasif = IncrementalClassifer
m_params = [
 {'threshold':0.7,
  'classifier':HT,
  'params':{'grace_period':10,'max_depth':4}},
  {'threshold':0.7,
  'classifier':no_change,
  'params':{}},
  {'max_micro_clusters':10}]
methods = [incClasif,incClasif2,clustream]



In [9]:
from streams.utils import FL, FU, generate_stream_section
from streams.stream_section import StreamSection
from river import metrics
from multiprocessing.pool import ThreadPool
from constants import NUMBER_OF_THREADS
import numpy as np

def get_most_frequent(predictions):
    ''' 
    Get the most common predction

    Parameters
    ----------
    predictions: list
        The list of predicted labels

    Returns
    ---------
    int
        The most popular class

    '''
    return max(set(predictions), key=predictions.count)


def update_performance_measures(predictions, true_label, B, metrics):
    ''' 
    The prediction for each awaiting (for a label) instance is made every K iterations 

    Parameters
    ----------
    predictions: dict
        The dictionary of predctions 

    true_label: int
        The true class of an instance

    B: int
        Number of bins #TODO add somewhere the describtion

    metrics: dict
        The dictionary holding the metrics to calculate the results
    '''
    interval = (len(predictions)-1)/B
    preds = list(predictions.values())
    for b in range(B+2):
        if b == 0:
            metrics[b].update(true_label, preds[0])
        elif b == B+1:
            metrics[b].update(true_label, preds[-1])
        if preds[int(1+interval*(b-1)):int((1+interval*b))]: #interval not empty
                y_pred = get_most_frequent(
                    preds[int(1+interval*(b-1)):int((1+interval*b))])
                metrics[b].update(true_label, y_pred)


def make_prediction_for_awaiting(h, cur_idx, P, L, K):
    ''' 
    The prediction for each awaiting (for a label) instance is made every K iterations 

    Parameters
    ----------
    h: classifier
        The trained classifer that predicts the class

    cur_idx: int
        The index of a currently processed observation

    P: dict
        The dictionary with predictions 

    L: dict
        The dictionary with awaiting results

    K: int
        The number indicates how often the predction is made

    '''
    # foreach instance in L add predictionin time cur_idx in P
    for idx, instance in L.items():
        # make prediction every K instances
        if abs(idx-cur_idx) % K == 0:
            P[idx][cur_idx] = h.predict_one(instance)


def add_delay_constant(stream, delay, no_delete_period, dataset_name, q):
    new_stream = []
    i = 0
    for idx_1, idx_2, x, y, in stream:
        if i < no_delete_period:
            new_stream.append((i, i, x, y))
            i = i+1
            continue
        if (i-no_delete_period) % delay == 0 and i-no_delete_period != 0:
            i += delay
        new_stream.append((i, i, x, None))
        new_stream.append((i+delay, i, x, y))
        i += 1
    new_stream.sort(key=lambda x: x[0])
    return StreamSection(f'{dataset_name}_delay_{q[0]}_{q[1]}', new_stream, False)

def add_delay_random(stream, delay, no_delete_period, dataset_name, q):
    new_stream = []
    used_indexes = []
    i = 0
    for idx_1, idx_2, x, y, in stream:
        if i < no_delete_period:
            new_stream.append((i, i, x, y))
            i = i+1
            continue
        while i in used_indexes:
            i+=1
        delay = np.random.randint(1,20)
        while i+delay in used_indexes:
            delay+=1 # if sampling again infinite loop possible
        new_stream.append((i, i, x, None))
        new_stream.append((i+delay, i, x, y))
        used_indexes.extend([i,i+delay])
        i += 1
    new_stream.sort(key=lambda x: x[0])
    return StreamSection(f'{dataset_name}_delay_{q[0]}_{q[1]}', new_stream, False)

def generate_streams(datastream, dataset_name, q, probas, delay_type, delay, warm_up_period):
    stream_set = []  # TODO chcek where this should be placed
    initial_stream = generate_stream_section(
        datastream, f'{dataset_name}_init_{q[0]}_{q[1]}', dataset_name, q[0], q[1])
    stream_set.append(initial_stream)
    if delay_type == 1:
        initial_stream = add_delay_constant(
            initial_stream.stream, delay, warm_up_period, dataset_name, q)
        dataset_name += '_constant_delay'
    elif delay_type == 2:
        initial_stream = add_delay_random(
            initial_stream.stream, delay, warm_up_period, dataset_name, q)
        dataset_name += '_random_delay'
        stream_set.append(initial_stream)

    for p in probas:
        ssl_stream = StreamSection(f'{dataset_name}_ssl_{p}_{q[0]}_{q[1]}', FU(
            initial_stream.stream, p, warm_up_period), False)
        lfs_stream = StreamSection(
            f'{dataset_name}_lfs_{p}_{q[0]}_{q[1]}', FL(ssl_stream.stream), True)
        stream_set.append(ssl_stream)
        stream_set.append(lfs_stream)
    return stream_set



def train_for_stream( my_stream,methods, methods_params,methods_name,
                       metric_fun, K, B,warm_up_period):
    results = {}
    # how many labelled instances appered -> needed for prediction of awaiting examples 
    labelled_insances_cnt = 0
    print(my_stream.__name__)
    for mi, method in enumerate(methods):
            # initilaze method and variables
        m = method(**methods_params[mi])
        metrics = [metric_fun() for _ in range(B+2)]
        h = m
        L = {}
        P = {}

        for cur_idx, init_idx, x, y in my_stream.stream:
            # TODO: can it be in this place
           
            # unlabelled instance
            if y is None:
                # add instnace and index
                L[cur_idx] = x
                P[cur_idx] = {}
                P[cur_idx][cur_idx] = h.predict_one(x)
                # TODO: think what to do if the method cannot deal with unlabelled
                h = h.learn_one(x)

            # labelled instance
            else:
                labelled_insances_cnt+=1
                if cur_idx != init_idx and init_idx in P.keys():  # delayed label
                    P[init_idx][cur_idx] = h.predict_one(x)
                    L.pop(init_idx)

                    update_performance_measures(
                        P[init_idx], y, B, metrics)
                # TODO: probably need to implment a better option of evaluation-> for now test then train is used
                else: #TODO: dopytac sie czy tojest ok 
                    if m._timestamp > warm_up_period:  # if in warmup period the prediction cannot be made
                        # if it was not delayed only last prediction exists
                        metrics[B+1].update(y, h.predict_one(x))
                
                make_prediction_for_awaiting(h, labelled_insances_cnt, P, L, K) 
                h = h.learn_one(x, y)
                
        results[methods_name[mi]] = metrics
    return my_stream.__name__,results

# K how many new labelled instances need to arrive before the new prediction is made
def train_and_evaluate(datastream, dataset_name, Q, probas, methods, methods_params,methods_name,
                       metric_fun, delay_type, K, B, delay, warm_up_period=10):
    '''
    Main evaluation and traing function
    delay_type - 0 - NONE, 1 - equal, 2- random
    '''
    pool = ThreadPool(NUMBER_OF_THREADS)
    results = {}
    for q in Q:
        # preparing streams part
        stream_set = generate_streams(
            datastream, dataset_name, q, probas, delay_type, delay, warm_up_period)

        # train and evaluation part
        results_for_q = pool.map(lambda my_stream: train_for_stream( my_stream,methods, methods_params, methods_name,
                       metric_fun, K, B,warm_up_period),stream_set )
    
            
        results[q] = results_for_q
    return results

        


In [10]:
from collections import Counter, defaultdict
import math
import typing
from river.cluster.clustream import CluStream
from river.cluster.clustream import CluStreamMicroCluster


class CluStreamMicroClusterWithLabel(CluStreamMicroCluster):
    """ 
    Extension of CluStreamMicroCluster class


    Parameters
    ----------
        labels: dict
            dict with number of each label occurnace

        the rest of parameters as in CluStreamMicroCluster class

    """
    def __init__(   self,   x: dict = defaultdict(float),
        labels: dict = {},
        w: float = None,
        timestamp: int = None ):
            super().__init__(x,w,timestamp)
            self.labels = labels
                
    def _add_label(self,y):
        """ add the label y occurance"""
        self.labels[y] = self.labels.get(y,0)+1

    def insert(self, x,y, w, timestamp):
        """insert new instance to a microcluster"""
        self.var_time.update(timestamp, w)
        for x_idx, x_val in x.items():
            self.var_x[x_idx].update(x_val, w)
        self._add_label(y)




class CluserAndLabel(CluStream):
    """ 
    Extension of CluStream class
        - instead of CluStreamMicroCluster a CluStreamMicroClusterWithLabel used
        - modified  _maintain_micro_clusters, predict_one, learn_one
        - macroclusters not used

    Parameters
       ----------
        train_period: int
            number of labelled instances the classifier needs to get before it can work with unlabelled ones

        the rest of parameters as in CluStream class

       """

    def __init__(self,train_period=0,
                 n_macro_clusters: int = 5, #TODO raczej usunac jesli nie potrzebne
                 max_micro_clusters: int = 100,
                 micro_cluster_r_factor: int = 2,
                 time_window: int = 1000,
                 time_gap: int = 100,
                 seed: int = None,
                 **kwargs, ):
        super().__init__(n_macro_clusters, max_micro_clusters,
                         micro_cluster_r_factor, time_window, time_gap, seed, **kwargs)
        self.micro_clusters: typing.Dict[int,
                                         CluStreamMicroClusterWithLabel] = {}
        self.train_period=train_period


    def _merge_clusters_label_count(self, labels1, labels2):
        """
        When two clusters are merged their labels dictionary also need to be merged
        and if they have the same keys, the value need to be summed

        Parameters
        ----------
        labels1,labels2: dict
            dictionaries of labels as keys and their frequency as values

        Returns
        -------
        dict
            with sum of each label frequency
        """

        cnt1 = Counter(labels1)
        cnt2 = Counter(labels2)
        return dict(cnt1 + cnt2)

    def _maintain_micro_clusters(self, x, w, y):
        """
        Diffrence in merging introduced
        TODO: better understandf what w does
        """
        # Calculate the threshold to delete old micro-clusters
        threshold = self._timestamp - self.time_window

        # Delete old micro-cluster if its relevance stamp is smaller than the threshold
        del_id = None
        for i, mc in self.micro_clusters.items():
            if mc.relevance_stamp(self.max_micro_clusters) < threshold:
                del_id = i
                break

        if del_id is not None:
            self.micro_clusters[del_id] = CluStreamMicroClusterWithLabel(
                x=x,
                w=w,
                labels={y: 1},
                timestamp=self._timestamp,
            )
            return

        # Merge the two closest micro-clusters
        closest_a = 0
        closest_b = 0
        min_distance = math.inf
        for i, mc_a in self.micro_clusters.items():
            for j, mc_b in self.micro_clusters.items():
                if i <= j:
                    continue
                dist = self._distance(mc_a.center, mc_b.center)
                if dist < min_distance:
                    min_distance = dist
                    closest_a = i
                    closest_b = j

        # diffrent merging - also labels count needs to be added
        labels_merged = self._merge_clusters_label_count(self.micro_clusters[closest_a].labels,
                                                         self.micro_clusters[closest_b].labels)
        self.micro_clusters[closest_a] += self.micro_clusters[closest_b]
        self.micro_clusters[closest_a].labels = labels_merged
        self.micro_clusters[closest_b] = CluStreamMicroClusterWithLabel(
            x=x,
            w=w,
            labels={y: 1},
            timestamp=self._timestamp,
        )

    # def return_microclusters(self):
    #     """ Method for printing the labels statistics in each microcluster"""
    #     for i, mc in self.micro_clusters.items():
    #         print(i,mc.labels)
    def sum_labels(self):
        """ Method for summing the number of stored labels -- needed for testing"""
        s = 0
        for i, mc in self.micro_clusters.items():
            s+=sum([v for v in mc.labels.values()])
        return s

    def predict_one(self, x):
        """ Prediction yields the majority class in a microcluser x belongs to """
        cluster_num = self._get_closest_mc(x)[0]
        labels = self.micro_clusters[cluster_num].labels
        return max(labels, key=labels.get)

    def learn_one(self, x, y=None, w=1):
        """ Learns y as well (orginal algorithm was an unsupervised one).
        If y not avaiable it predicts a label and assigns it as a pseudolabel
        The macroclusters are not used"""
        if y is None:
            if self._timestamp < self.train_period:
                return self
            y = self.predict_one(x)
        
        self._timestamp += 1

        if not self._initialized:
            self.micro_clusters[len(self.micro_clusters)] = CluStreamMicroClusterWithLabel(
                x=x,
                w=w,
                labels={y: 1},
                # When initialized, all micro clusters generated previously will have the timestamp reset to the current
                # time stamp at the time of initialization (i.e. self.max_micro_cluster - 1). Thus, the timestamp is set
                # as follows.
                timestamp=self.max_micro_clusters - 1,
            )

            if len(self.micro_clusters) == self.max_micro_clusters:
                self._initialized = True

            return self

            # Determine the closest micro-cluster with respect to the new point instance
        closest_id, closest_dist = self._get_closest_mc(x)
        closest_mc = self.micro_clusters[closest_id]

        # Check whether the new instance fits into the closest micro-cluster
        if closest_mc.weight == 1:
            radius = math.inf
            center = closest_mc.center
            for mc_id, mc in self.micro_clusters.items():
                if mc_id == closest_id:
                    continue
                distance = self._distance(mc.center, center)
                radius = min(distance, radius)
        else:
            radius = closest_mc.radius(self.micro_cluster_r_factor)

        if closest_dist < radius:
            closest_mc.insert(x, w, y, self._timestamp)
            return self

        # If the new point does not fit in the micro-cluster, micro-clusters
        # whose relevance stamps are less than the threshold are deleted.
        # Otherwise, closest micro-clusters are merged with each other.
        self._maintain_micro_clusters(x=x, w=w, y=y)
        return self


In [11]:
a

NameError: name 'a' is not defined

 # Experiments

In [16]:
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")

In [17]:
date_time

'27_03_2023_19_51_28'

In [9]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2023-03-27 19:49:18.721127'

In [5]:
stream_set = generate_streams(
            hp, dataset_name, (0,10000), probas, 0, delay, 10)

NameError: name 'generate_streams' is not defined

In [11]:
for my_stream in stream_set:
    print(train_for_stream( my_stream, methods,m_params,['HT','Dummy','CluStreamAndLabel'],
                       Accuracy,5,10,True,10))

NameError: name 'stream_set' is not defined

In [3]:
res = train_and_evaluate(hp,dataset_name,Q,probas,methods,m_params,['HT','Dummy','CluStreamAndLabel'],Accuracy,2,10,5,10,True)

In [19]:
ts = res[(0, 1000)][0][1]['HT']

'0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8624873609706775'

In [6]:
os.listdir('logs')

['27_03_2023_19_55_43.log',
 '27_03_2023_20_25_57.log',
 '27_03_2023_20_27_51.log',
 '27_03_2023_20_28_36.log',
 '27_03_2023_20_33_53.log',
 'date_time.log']

In [12]:
df = pd.read_csv('logs\\27_03_2023_20_33_53.log',header=None)

In [16]:
df.loc[0,0]

'27-Mar-23 20:33:53-Electricity_init_0_1000-HT-[0.0'

In [11]:
[t.get()for t in ts]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8685540950455005]

In [5]:
['Accuracy': 0.00%,
 'Accuracy': 0.00%,
 'Accuracy': 0.00%,
 'Accuracy': 0.00%,
 'Accuracy': 0.00%,
 'Accuracy': 0.00%,
 'Accuracy': 86.15%]

SyntaxError: invalid syntax (2575889801.py, line 1)

In [4]:
train_and_evaluate(hp,dataset_name,Q,probas,[clustream],[{'max_micro_clusters':5}],['CluStreamAndLabel'],Accuracy,0,10,5,10,False)

HyperPlane_init_0_10000HyperPlane_ssl_0.1_0_10000
HyperPlane_lfs_0.1_0_10000



{(0,
  10000): [('HyperPlane_init_0_10000',
   {'CluStreamAndLabel': [Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 52.29%]}), ('HyperPlane_ssl_0.1_0_10000',
   {'CluStreamAndLabel': [Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 52.33%]}), ('HyperPlane_lfs_0.1_0_10000',
   {'CluStreamAndLabel': [Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 0.00%,
     Accuracy: 52.30%]})]}

In [ ]:
from river import cluster
from river import stream
# X = [ [1, 2],[1, 4],[1, 0],[-4, 0],[-4, 4]]

# Y = [1,1,2,3,1]
#incr = IncrementalClassifer(**params)
Ht = GaussianNB()
led_stream = get_LED()
for i,(x,y) in enumerate(led_stream):
  
    if i <9990:
        # if y ==7:
        #     print(clustream.return_microclusters() )
        if i %10 == 9:
            Ht = Ht.learn_one(x,y)
        else:
            Ht = Ht.learn_one(x = x,y = y)
        #print(clustream.sum_labels())

    else:
        print(Ht.predict_one(x),y)



2 8
2 0
2 8
2 0
2 9
2 7
2 2
2 0
2 7
2 4
2 2
2 9
2 7
2 1
2 7
2 6
2 0
2 9
2 2
2 6
2 8
2 2
2 6
2 8
2 5
2 6
2 4
2 7
2 7
2 1
2 3
2 0
2 9
2 6
2 9
2 7
2 3
2 7
2 6
2 4
2 8
2 2
2 9
2 7
2 0
2 8
2 0
2 0
2 7
2 4
2 7
2 5
2 4
2 1
2 3
2 2
2 9
2 6
2 5
2 3
2 6
2 7
2 7
2 7
2 6
2 7
2 3
2 4
2 9
2 5
2 0
2 7
2 6
2 1
2 1
2 8
2 5
2 7
2 0
2 5
2 7
2 8
2 7
2 6
2 1
2 8
2 5
2 8
2 7
2 1
2 5
2 1
2 9
2 1
2 0
2 1
2 4
2 0
2 3
2 1
2 8
2 3
2 7
2 9
2 4
2 8
2 9
2 6
2 6
2 9
2 2
2 0
2 7
2 9
2 5
2 1
2 3
2 7
2 2
2 7
2 1
2 6
2 1
2 2
2 2
2 5
2 7
2 3
2 0
2 2
2 2
2 8
2 9
2 0
2 8
2 9
2 7
2 1
2 0
2 2
2 5
2 5
2 3
2 4
2 0
2 2
2 3
2 4
2 5
2 5
2 2
2 4
2 6
2 3
2 7
2 1
2 9
2 0
2 5
2 1
2 8
2 2
2 3
2 7
2 6
2 6
2 1
2 9
2 4
2 4
2 5
2 3
2 7
2 1
2 3
2 6
2 8
2 1
2 2
2 1
2 5
2 0
2 8
2 6
2 4
2 7
2 7
2 4
2 4
2 1
2 2
2 9
2 3
2 2
2 3
2 7
2 8
2 4
2 0
2 3
2 9
2 7
2 8
2 9
2 3
2 4
2 8
2 9
2 2
2 1
2 7
2 5
2 3
2 5
2 6
2 6
2 6
2 4
2 4
2 1
2 2
2 5
2 8
2 4
2 9
2 7
2 6
2 2
2 3
2 7
2 2
2 7
2 6
2 1
2 3
2 1
2 7
2 5
2 6
2 8
2 4
2 5
2 2
2 9
2 1
2 4
2 6
2 7
2 9
2 2


KeyboardInterrupt: 

In [ ]:
pool = ThreadPool(5)

In [ ]:
l0 = get_LED(0.1,True,0)
l1 = get_LED(0.1, True,3)
l2 = get_LED(0.1, True,5)
l3 = get_LED(0.1,True,7)

In [ ]:
ds = ConceptDriftStream(ConceptDriftStream(ConceptDriftStream(l0,l1,position = 3750),l2,position = 7500),l3,position=15000)

In [ ]:
res = {}
for  dataset_name,data in tqdm({'HyperPlane':get_Hyperplane()}.items()):
    print(dataset_name)
    res[dataset_name] = train_and_evaluate(data,dataset_name,Q,probas,methods,m_params,['HT','NoChange','CluStreamAndLabel'],Accuracy,2,10,5,100)
    

  0%|          | 0/1 [00:00<?, ?it/s]

HyperPlane
HyperPlane_init_0_100000HyperPlane_delay_0_100000
HyperPlane_random_delay_ssl_0.1_0_100000

HyperPlane_random_delay_lfs_0.1_0_100000


100%|██████████| 1/1 [11:08:49<00:00, 40129.02s/it]


In [ ]:
res

{'HyperPlane': {(0,
   100000): [('HyperPlane_init_0_100000',
    {'HT': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 80.82%],
     'NoChange': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 50.26%],
     'CluStreamAndLabel': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 57.61%]}), ('HyperPlane_delay_0_100000',
    {'HT': [Accuracy: 79.42%,
      Accuracy: 0.00%,
      Accuracy: 82.97%,
      Accuracy: 79.62%,
      Accuracy: 82.97%,
      Accuracy: 79.42%,
      Accuracy: 79.42%],
     'NoChange': [Accuracy: 50.05%,
      Accuracy: 0.00%,
      Accuracy: 55.02%,
      Accuracy: 49.98%,
      Accuracy: 52.84%,
      Accuracy: 50.26%,
      Accuracy: 50.26%],
     'CluStreamAndLabel'

In [ ]:
res

{'Airlines': {(0,
   10000): [('Airlines_init_0_10000',
    {'HT': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 56.78%],
     'NoChange': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 56.90%]}), ('Airlines_delay_0_10000',
    {'HT': [Accuracy: 55.77%,
      Accuracy: 0.00%,
      Accuracy: 46.67%,
      Accuracy: 55.19%,
      Accuracy: 46.67%,
      Accuracy: 55.56%,
      Accuracy: 55.56%],
     'NoChange': [Accuracy: 54.88%,
      Accuracy: 0.00%,
      Accuracy: 36.67%,
      Accuracy: 54.00%,
      Accuracy: 66.67%,
      Accuracy: 56.01%,
      Accuracy: 56.01%]}), ('Airlines_random_delay_ssl_0.1_0_10000',
    {'HT': [Accuracy: 57.41%,
      Accuracy: 0.00%,
      Accuracy: 100.00%,
      Accuracy: 56.48%,
      Accuracy: 100.00%,
      Accuracy: 57.05%,
      Accuracy: 57.05%

In [ ]:
res

{'Airlines': {(0,
   10000): [('Airlines_init_0_10000',
    {'HT': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 57.71%],
     'NoChange': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 56.90%]}), ('Airlines_constant_delay_ssl_0.1_0_10000',
    {'HT': [Accuracy: 55.69%,
      Accuracy: 54.93%,
      Accuracy: 55.48%,
      Accuracy: 54.90%,
      Accuracy: 55.49%,
      Accuracy: 55.23%,
      Accuracy: 55.28%],
     'NoChange': [Accuracy: 51.17%,
      Accuracy: 53.33%,
      Accuracy: 53.51%,
      Accuracy: 52.83%,
      Accuracy: 52.55%,
      Accuracy: 53.88%,
      Accuracy: 56.08%]}), ('Airlines_constant_delay_lfs_0.1_0_10000',
    {'HT': [Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Ac

In [ ]:
params = {'threshold': 0.7,
 'classifier': river.tree.hoeffding_adaptive_tree_classifier.HoeffdingAdaptiveTreeClassifier,
 'params': {'grace_period': 10, 'max_depth': 4}}

In [ ]:
from river.datasets import synth

In [ ]:
synth.Agrawal()

In [ ]:
df

,0,1,2,3,4,5,6,7,8
0,-1.534156,-0.130580,-0.244595,-0.002791,-1.136166,-1.551956,1.024612,0.250979,-0.606804
1,1.159820,-0.726918,-1.149038,-0.700544,0.086833,1.554286,-1.372045,-0.917997,-1.186537
2,1.341117,-0.726918,1.055541,-1.398296,1.659260,0.777725,-0.983647,-0.567304,0.769517
3,-1.366826,-0.247671,0.659847,-0.700544,-0.961452,-1.551956,0.786553,-0.216611,0.223482
4,-0.816778,1.899359,1.055541,-0.002791,0.086833,1.554286,-1.372045,-1.034894,1.076672
...,...,...,...,...,...,...,...,...,...
995,0.915641,-0.726918,1.620818,1.392714,-1.310880,0.001165,-0.191258,-1.502485,1.299967
996,-0.322038,1.313746,-0.470706,-0.700544,-0.786738,0.777725,-0.875194,-1.619382,1.011343
997,-0.784605,1.865074,-1.488204,0.694962,-0.262595,0.389445,-0.717107,1.186160,0.998029
998,0.559457,-0.726918,1.677346,1.392714,-1.136166,-0.775396,1.549851,-1.502485,-0.423991


In [ ]:
knn = KNN(1)

In [ ]:
labels.index

Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64', length=1000)

In [ ]:
agrawal = get_AGRAWL()
df = pd.DataFrame()
labels = pd.Series()
for i,(x,y) in enumerate(agrawal.take(1000)):
    df = pd.concat([df, pd.DataFrame(x, index = [i])])
    labels = pd.concat([labels,pd.Series([y],index=[i])])

    

C:\Users\gosia\AppData\Local\Temp\ipykernel_16612\764246324.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  labels = pd.Series()


In [ ]:
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df))
df.loc[:,'label'] = labels

C:\Users\gosia\AppData\Local\Temp\ipykernel_16612\883775822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L.drop(columns='label',inplace=True)
C:\Users\gosia\AppData\Local\Temp\ipykernel_16612\883775822.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  U.drop(columns='label',inplace=True)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def _collect_observation(observations,labels, x, y):
    observations = pd.concat([observations, pd.DataFrame(x, index = [i])])
    labels = pd.concat([labels,pd.Series([y],index=[i])])
    return observations,labels

In [ ]:
def prepare_sets(observations,labels):
    scaler = StandardScaler()
    observations = pd.DataFrame(scaler.fit_transform(observations))
    observations.loc[:,'label'] = labels
    L = observations[observations['label']==1]
    U = observations[observations['label'] == 0]
    L.drop(columns='label',inplace=True)
    L.reset_index(inplace = True, drop=True)
    U.drop(columns='label',inplace=True)
    U.reset_index(inplace = True,drop=True)

    for _,x in L.iterrows():
        knn = knn.learn_one(x,1)
    return L,U

In [ ]:
def _label_the_closest(L,U,knn):
    min_ind = np.argmin(np.array(U.apply(lambda x: knn._nn.find_nearest((x,None))[0][1],axis=1 )))
    knn = knn.learn_one(x,1)
    L = pd.concat([L,pd.DataFrame(U.iloc[min_ind]).T])
    U = U.drop(index=min_ind)
    U.reset_index(inplace=True,drop = True)
    return L,U

In [ ]:
requested_ratio = 0.75
knn = KNNClassifier(n_neighbors=1)
while(len(L)/(len(L)+len(U))<requested_ratio):
    L,U = _label_the_closest(L,U)


In [ ]:
L,U = prepare_sets(df,labels)

C:\Users\gosia\AppData\Local\Temp\ipykernel_16612\3541668259.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L.drop(columns='label',inplace=True)
C:\Users\gosia\AppData\Local\Temp\ipykernel_16612\3541668259.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  U.drop(columns='label',inplace=True)


In [ ]:
class oneNNClassifer():
    def __init__(self, training_time = 1000, threshold = 1,positive =1,negative = 0,required_ratio = 0.75) -> None:
        self.threshold = threshold
        self._timestamp = 0
        self.training_time =training_time
        self.L = pd.DataFrame()
        self.U = pd.DataFrame()
        self.positive = positive # which class is considered to be positive 
        self.negative = negative
        self.scaler = StandardScaler() #TODO: check how the river scaler is working and consider swapping it somehow
        self.classifier = riverKNN(n_neighbors=1)
        self.required_ratio = required_ratio #TODO: maybe diffrent stopping criteria

    def _label_the_closest(self,L,U):
        min_ind = np.argmin(np.array(U.apply(lambda x: self.classifier._nn.find_nearest((x,None))[0][1],axis=1 )))
        self.classifier = self.classifier.learn_one(x,1)
        L = pd.concat([L,pd.DataFrame(U.iloc[min_ind]).T])
        U = U.drop(index=min_ind)
        U.reset_index(inplace=True,drop = True)
        return L,U

    def _train_classifier(self):
        scaler.fit(pd.concat([self.L,self.U]))
        self.L = scaler.transform(L)
        self.U = scaler.transform(U)
        for _, x in L.iterrows(): # TODO: think how to avoid loop
            self.classifier = self.classifier.learn_one(x,1)
            
        while (len(L)/(len(L)+len(U)))<self.required_ratio:
            self.L,self.U = self._label_the_closest(L,U)

    def learn_one(self,x,y):
        if self._timestamp < self.training_time:
            if y == self.positive:
                self.L = pd.concat([self.L, pd.DataFrame(x, index = [self._timestamp])])
            else:
                self.U = pd.concat([self.U, pd.DataFrame(x, index = [self._timestamp])])

        elif self._timestamp == self.training_time:  
            self._train_classifier()

        else:
            self.classifier = self.classifier.learn_one(x,int(y==self.positive))
        self._timestamp+=1
        return self

    def predict_one(self,x):
        x = self.scaler.transform(x)
        dist =  self.classifier._nn.find_nearest((x,None))[0][1]
        if dist < self.threshold:
            return self.positive
        return self.negative



In [ ]:
np.array(x)

array([2.34179867, 2.22486556, 2.68454011, 2.2700705 , 1.86016665,
       1.87065962, 2.52084008, 3.25938943, 5.38527136, 1.73833883,
       2.00255849, 1.15456905, 2.76046625, 4.18231088, 1.33047636,
       1.59439158, 1.14608507, 6.20053973, 2.04137088, 1.74918822,
       1.61957739, 2.2752477 , 1.58551992, 3.40846331, 4.06817461,
       2.51197602, 3.40460803, 1.70680428, 2.66166373, 3.56394397,
       2.7866158 , 2.77304289, 1.82548624, 1.36528962, 1.29763871,
       2.19429105, 1.32897383, 1.38850589, 2.28629292, 1.96037832,
       0.73493478, 1.59655178, 1.97029011, 1.2878852 , 1.93113636,
       1.80869179, 3.19021139, 3.49239072, 1.64296473, 1.97155481,
       1.73540989, 3.72590951, 0.66382453, 3.21297733, 1.32537109,
       1.3717631 , 2.27007875, 2.34377458, 6.10969098, 1.94372621,
       3.20147471, 2.37562192, 1.57101392, 3.10966908, 2.36258745,
       2.51707758, 2.26579243, 1.90474872, 2.08092043, 2.76671469,
       2.46729485, 1.50640372, 1.93326275, 2.26385497, 3.83724

In [ ]:
gen = synth.ConceptDriftStream(stream=synth.LEDDrift(123,0,n_drift_features=1),
                          drift_stream=synth.LEDDrift(123,0,n_drift_features=4),
                            seed=1, position=500, width=50)
# Take 1000 instances from the infinite data generator
dataset = iter(gen.take(1000))
model = tree.HoeffdingAdaptiveTreeClassifier(
     grace_period=100,
     delta=1e-5,
     leaf_prediction='nb',     nb_threshold=10,
     seed=0
 )
metric = metrics.Accuracy()
for x,y in dataset:
    model = model.learn_one(x,y)

In [ ]:
for x,y in iter(gen.take(10)):
    print(x,y,model.predict_one(x))


{0: 1, 1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 1} 0 8
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1} 5 8
{0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0} 1 8
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1} 2 8
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1} 5 8
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1} 8 8
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1} 8 8
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1} 5 8
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1} 5 8
{0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0} 7 8


# DATASETS

In [ ]:
fligths = [(dict((i,x[i]) for i in x.keys() if i!='Delay'),x['Delay']) for x,_ in stream.iter_arff('../../datasets/airlines.arff.zip', compression='infer')]
lda = datasets.synth.LEDDrift(123,0,True,0)
electricity = [(dict((i,x[i]) for i in x.keys() if i!='class'),x['class']=='UP') for x,_ in stream.iter_arff('../../datasets/elecNormNew.arff.zip', compression='infer')]

In [ ]:
datasets.synth.LEDDrift()

In [ ]:
Q = [(0,1000),(1000,2000),(2000,3000),(3000,4000),(4000,5000)]
datastream = lda
P = [0.1,0.2,0.5,0.6]
dataset_name = 'LED'
stream_set = []

train_and_evaluate(datastream,dataset_name,Q,P)

In [ ]:
[x.__name__ for x in stream_set]

['Cover_Type_init_0_1000',
 'Cover_Type_ssl_0.1_0_1000',
 'Cover_Type_lfs_0.1_0_1000',
 'Cover_Type_ssl_0.2_0_1000',
 'Cover_Type_lfs_0.2_0_1000',
 'Cover_Type_ssl_0.5_0_1000',
 'Cover_Type_lfs_0.5_0_1000',
 'Cover_Type_ssl_0.6_0_1000',
 'Cover_Type_lfs_0.6_0_1000',
 'Cover_Type_init_1000_2000',
 'Cover_Type_ssl_0.1_1000_2000',
 'Cover_Type_lfs_0.1_1000_2000',
 'Cover_Type_ssl_0.2_1000_2000',
 'Cover_Type_lfs_0.2_1000_2000',
 'Cover_Type_ssl_0.5_1000_2000',
 'Cover_Type_lfs_0.5_1000_2000',
 'Cover_Type_ssl_0.6_1000_2000',
 'Cover_Type_lfs_0.6_1000_2000',
 'Cover_Type_init_2000_3000',
 'Cover_Type_ssl_0.1_2000_3000',
 'Cover_Type_lfs_0.1_2000_3000',
 'Cover_Type_ssl_0.2_2000_3000',
 'Cover_Type_lfs_0.2_2000_3000',
 'Cover_Type_ssl_0.5_2000_3000',
 'Cover_Type_lfs_0.5_2000_3000',
 'Cover_Type_ssl_0.6_2000_3000',
 'Cover_Type_lfs_0.6_2000_3000',
 'Cover_Type_init_3000_4000',
 'Cover_Type_ssl_0.1_3000_4000',
 'Cover_Type_lfs_0.1_3000_4000',
 'Cover_Type_ssl_0.2_3000_4000',
 'Cover_Type_lfs

{'threshold': 0.7,
 'classifier': river.tree.hoeffding_adaptive_tree_classifier.HoeffdingAdaptiveTreeClassifier,
 'params': {'grace_period': 100, 'max_depth': 4}}

In [ ]:
for q in stream_set:
    clustream = CluserAndLabel(4,
         max_micro_clusters=7,
         time_gap=3,
         seed=0,
         halflife=0.4 )
    for i,(x,y) in enumerate(q.stream):
        print(q.__name__)
        if i <990:
            clustream = clustream.learn_one(x = x,y = y)
            #print(clustream.sum_labels())

        else:
            print(clustream.predict_one(x),y)

    #clustream.return_microclusters() 

LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init

KeyboardInterrupt: 

In [ ]:
i

999

In [ ]:
hp = datasets.synth.RandomRBFDrift()
for i in hp.take(10):
    print(i)

({0: 0.4474707370607147, 1: 0.4112430647339043, 2: 0.5774887345532604, 3: 0.9345089494963857, 4: 0.625470816542423, 5: 0.9528893392318104, 6: 0.23390913685064327, 7: 0.2632706539603502, 8: 0.6487674488470405, 9: 0.3223666205525}, 0)
({0: 0.38615946253321076, 1: 0.40767762106825467, 2: 0.8820781268130979, 3: 1.0294262479340588, 4: 0.16964208338080689, 5: 0.4068516375090169, 6: 0.920811978379108, 7: 0.31470618207032774, 8: 0.2904818353582927, 9: 0.5318226376467426}, 1)
({0: 0.7407199640645749, 1: 0.675852803052017, 2: 0.09199640007669287, 3: 0.6597045689218738, 4: 0.9128605755224982, 5: 0.6715464514966379, 6: 0.5001450370818181, 7: 0.2822393064383389, 8: 0.9402458315878417, 9: 0.2657905405040588}, 0)
({0: 0.41429299227040856, 1: 0.40059196758918525, 2: 0.621787171747695, 3: 0.9122077031232282, 4: 0.6161816448475379, 5: 0.9673529227812119, 6: 0.24849079855787165, 7: 0.22698910907704506, 8: 0.6177538662843673, 9: 0.32919829542592816}, 0)
({0: 0.033234076182812905, 1: 0.5225248193493289, 2:

In [ ]:
model = tree.HoeffdingTreeClassifier(    grace_period=100,    delta=1e-5)

In [ ]:
metric = metrics.Accuracy()
evaluate.progressive_val_score(led_stream,model,metric)

Accuracy: 10.71%